In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
mp_drawing = mp.solutions.drawing_utils #my notes=  to visulize the lines on the body 
mp_pose = mp.solutions.pose # from solutions we want the pose estemation model

In [22]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### calculate the angle 

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) #y2-y1 , x2-x1 , here we calculated the dgree in radians
    angle = np.abs(radians*180.0/np.pi) # we converted radiance to dgree
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

#### left shoulder abduction

In [11]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0 
stage = None #down or up

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle > 170:
                stage = "up"
            if angle < 30 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2


##### left shoulder abduction- ill just change how the reps are calculated

In [16]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)
            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                
                       
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3


##### left shoulder abduction- ill just change how the reps are calculated - the user will enter how many reps he want + motivitinal phrase at the end 

In [19]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       print("Great job! You smashed your goal 💪")
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1
2
Great job! You smashed your goal 💪


note: same code just the motivinal phrase will be printed on the screen

In [21]:
import cv2
import mediapipe as mp
import numpy as np
import time

cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       cv2.putText(image, "Great job! You smashed your goal", 
                                   (50, 200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 
                                   1, (0, 0, 255), 3, cv2.LINE_AA)
                       cv2.imshow('Mediapipe Feed', image)
                       cv2.waitKey(1)
                       time.sleep(5)
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2


with a voice

In [20]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Text-to-speech engine
engine = pyttsx3.init()

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       message = "Great job! You smashed your goal"
                       cv2.putText(image, message ,
                                   (50, 200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 
                                   1, (0, 0, 255), 3, cv2.LINE_AA)
                       cv2.imshow('Mediapipe Feed', image)
                       cv2.waitKey(1)
                       engine.say(message)
                       engine.runAndWait()
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1
2


same code as above but with alert

In [7]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

cap = cv2.VideoCapture(0)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # Visualize shoulder angle
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Alert if elbow is not straight
            if elbow_angle < 160:
                alert_text = "Straighten your elbow"
                cv2.putText(image, alert_text,
                            (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1, (0, 0, 255), 3, cv2.LINE_AA)
                engine.say(alert_text)
                engine.runAndWait()

            # Shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(counter)

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        cv2.putText(image, message,
                                    (50, 200), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1)
                        engine.say(message)
                        engine.runAndWait()
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))              
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


fixing the warning 

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

cap = cv2.VideoCapture(0)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False  # Track elbow alert status

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # Visualize shoulder angle
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Dynamic elbow alert
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    cv2.putText(image, alert_text,
                                (50, 150),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 0, 255), 3, cv2.LINE_AA)
                    engine.say(alert_text)
                    engine.runAndWait()
                    elbow_warning_issued = True
                else:
                    cv2.putText(image, "Elbow bent",
                                (50, 150),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(counter)

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        cv2.putText(image, message,
                                    (50, 200), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1)
                        engine.say(message)
                        engine.runAndWait()
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))              
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    speak(alert_text)
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction rep counter
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Exception in thread Thread-5 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20232\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 1
Rep: 2


Exception in thread Thread-8 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20232\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 3


Exception in thread Thread-10 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20232\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-12 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 4
